# Convert LabelMe Annotation to CSV-dataset

In [1]:
import json
import cv2 as cv
import numpy as np
import csv
import os
from os import walk
import random
from datetime import datetime

In [5]:
# Path the the folder containing the image annotations
path = '../../03 Data/Dataset2_onPallet/'
#path = '../../03 Data/Simple Dataset/'
#path = '../../03 Data/Single_Image/'
validation_percent = 0.2

#print(os.listdir(path))

all_files = os.listdir(path)

file_names = []
for file in all_files: 
    if file[-5:]  == '.json':
        file_names.append(file)
    
#print(file_names)
random.shuffle(file_names)
num_files = len(file_names)

#val_files = file_names[0:int(num_files*validation_percent)]
#train_files = file_names[int(num_files*validation_percent):]

#val_file = open('list_of_img_in_val_set_12-03.csv')
#val_file = open('list_of_img_in_OP_val_set_14-04.csv')
val_file = open('List_of _images_in_OP_rectified_val_set_04.txt')
val_files = list(val_file)
val_file.close()
for i, img in enumerate(val_files):
    val_files[i] = img.strip('.png\n')+'.json'

#train_file = open('list_of_img_in_train_set_12-03.csv')
#train_file = open('list_of_img_in_OP_train_set_14-04.csv')
train_file = open('List_of _images_in_OP_rectified_train_set_04.txt')
train_files = list(train_file)
train_file.close()
for i, img in enumerate(train_files):
    train_files[i] = img.strip('.png\n')+'.json'


print(val_files)
print(train_files)

print(len(file_names))
print(len(val_files))
print(len(train_files))

now = datetime.now()
date = now.strftime("%d-%m")

names = ['train_set_'+date+'.csv', 'val_set_'+date+'.csv']
combined_files = [train_files, val_files]
wh_list = []
# create csv of train-set and val-set for feeding to the network
for i in range(len(names)):
    with open(names[i], 'w', newline='') as open_file:
        writer = csv.writer(open_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        with open('list_of_img_in_'+names[i], 'w') as f:
            for file in combined_files[i]:
                with open(path + file) as json_file:
                    data = json.load(json_file)
                    image_name = data['imagePath']
                    f.write("%s\n" % image_name)
                        # check if image contains annotations.
                    if len(data['shapes']) != 0:
                        for annotation in data['shapes']:
                                rot_rect = cv.minAreaRect(np.float32(annotation['points']))
                                bp = cv.boxPoints(rot_rect) # bp = box points
                                #p1 = [rot_rect[0]+]
                                # make sure that width and height are consistent
                                if rot_rect[-2][0] < rot_rect[-2][1]:
                                    rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
                                # make sure that no angle is above 90 or below -90
                                wh_list.append(list(rot_rect[1]))
                                if rot_rect[-1] > 90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
                                if rot_rect[-1] < -90:
                                    rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
                                class_name = annotation['label']
                                writer.writerow([path + image_name, bp[0][0], bp[0][1], bp[1][0], bp[1][1],
                                                 bp[2][0], bp[2][1], bp[3][0], bp[3][1], class_name,rot_rect[-1]])
                                # format: img_path, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, class_name, angle
                    else:
                        writer.writerow([path + image_name, ",,,,,,,,,,"])
       

['OP_colorIMG_0.json', 'OP_colorIMG_2.json', 'OP_colorIMG_3.json', 'OP_colorIMG_4.json', 'OP_colorIMG_5.json', 'OP_colorIMG_6.json', 'OP_colorIMG_7.json', 'OP_colorIMG_8.json', 'OP_colorIMG_9.json', 'OP_colorIMG_10.json', 'OP_colorIMG_11.json', 'OP_colorIMG_12.json', 'OP_colorIMG_13.json', 'OP_colorIMG_14.json', 'OP_colorIMG_15.json', 'OP_colorIMG_16.json', 'OP_colorIMG_17.json', 'OP_colorIMG_18.json', 'OP_colorIMG_19.json', 'OP_colorIMG_20.json', 'OP_colorIMG_21.json', 'OP_colorIMG_22.json', 'OP_colorIMG_23.json', 'OP_colorIMG_24.json', 'OP_colorIMG_25.json', 'OP_colorIMG_26.json', 'OP_colorIMG_27.json', 'OP_colorIMG_28.json', 'OP_colorIMG_29.json', 'OP_colorIMG_30.json', 'OP_colorIMG_31.json', 'OP_colorIMG_32.json', 'OP_colorIMG_33.json', 'OP_colorIMG_34.json', 'OP_colorIMG_35.json', 'OP_colorIMG_36.json', 'OP_colorIMG_37.json', 'OP_colorIMG_38.json', 'OP_colorIMG_39.json', 'OP_colorIMG_40.json', 'OP_colorIMG_41.json', 'OP_colorIMG_42.json', 'OP_colorIMG_43.json', 'OP_colorIMG_44.jso

In [3]:
print(wh_list)
f = open("brick_wh.txt", 'w')
for width, height in wh_list:
    f.write("%f, %f \n" % (width, height))
f.close()    

[[165.8741455078125, 37.264923095703125], [167.12200927734375, 39.401580810546875], [163.13986206054688, 39.57955551147461], [168.87521362304688, 38.842594146728516], [168.71420288085938, 38.52050018310547], [168.61431884765625, 38.55280303955078], [164.4154052734375, 38.23244094848633], [185.17161560058594, 42.307090759277344], [186.69842529296875, 45.14986801147461], [138.46446228027344, 33.67230224609375], [145.73843383789062, 38.398277282714844], [143.3229522705078, 34.85377502441406], [182.19021606445312, 44.30568313598633], [181.59365844726562, 44.3056755065918], [170.2935333251953, 39.67049026489258], [167.90402221679688, 40.74193572998047], [170.5751953125, 42.900901794433594], [182.284423828125, 45.4871826171875], [180.6664276123047, 45.4871711730957], [181.49842834472656, 46.66145324707031], [182.3803253173828, 46.666622161865234], [182.239501953125, 43.12274169921875], [182.0291748046875, 43.12323760986328], [176.74359130859375, 45.486324310302734], [168.83413696289062, 40.7